In [228]:
import modin.pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import LabelEncoder
%matplotlib inline


encodings = ['a','n','o','~']

train = pd.read_csv("./data/train_signal.csv").drop(['ID'], axis=1)
init_test = pd.read_csv("./data/test_signal.csv")

le = LabelEncoder()

train['Type_new'] = le.fit_transform(train['Type'].astype(str))

X = train.drop(['Type','Type_new'], axis=1)
y = train['Type_new']


np.where(X.values >= np.finfo(np.float64).max)

X = X.fillna(train.mean())
test = init_test.drop("ID", axis=1)
test = test.fillna(test.mean())

print(X.isnull().sum().sum())



0


  Type     X0     X1     X2     X3     X4     X5     X6     X7     X8  ...  \
0    N -0.107 -0.100 -0.086 -0.078 -0.071 -0.057 -0.049 -0.035 -0.028  ...   
1    N  2.762  3.313  3.863  4.292  4.594  4.623  4.408  3.817  3.400  ...   
2    N -0.246 -0.200 -0.159 -0.125 -0.101 -0.090 -0.084 -0.078 -0.072  ...   
3    ~  0.519  0.778  1.073  1.392  1.672  1.895  2.012  2.023  1.946  ...   
4    ~  0.011 -0.103 -0.265 -0.371 -0.409 -0.422 -0.418 -0.411 -0.386  ...   

   X5991  X5992  X5993  X5994  X5995  X5996  X5997  X5998  X5999  Type_new  
0 -1.072 -1.028 -0.978 -0.912 -0.862 -0.804 -0.724 -0.630 -0.499         1  
1  0.003  0.148  0.241  0.310  0.345  0.368  0.397  0.426  0.438         1  
2 -0.038  0.032  0.107  0.165  0.194  0.194  0.159  0.119  0.072         1  
3 -0.052 -0.084 -0.099 -0.101 -0.090 -0.067  0.003  0.096  0.179         3  
4  0.829  0.763  0.481  0.126 -0.144 -0.224 -0.250 -0.222 -0.207         3  

[5 rows x 6002 columns]


In [229]:
import tensorflow as tf
import tensorflow.keras as keras # or import keras 

In [230]:
# Flatten the input
X_train = X_train.reshape(X_train.shape[0], -1)
X_test = X_test.reshape(X_test.shape[0], -1)

In [232]:
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
X = scaler.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

pca = PCA(n_components=0.95) # reduce dimensions to explain 95% of variance as recommended here: https://www.mikulskibartosz.name/pca-how-to-choose-the-number-of-components/
pca.fit(X_train)

X_train = pca.transform(X_train)
X_test = pca.transform(X_test)

print(X_train.shape) 
print(X_test.shape)

(8751, 750)
(4311, 750)


In [233]:
from keras.utils import to_categorical

print(y_train.shape, y_test.shape)

y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

print(y_train.shape, y_test.shape)

(8751,) (4311,)
(8751, 4) (4311, 4)


In [234]:
from keras.models import Sequential
from keras.layers import Dense, Activation

model = Sequential()
model.add(Dense(32, input_shape=(750,))) # first dense layer, 32 hidden units
model.add(Activation('relu'))            # activation layer
model.add(Dense(4))                     # second dense layer
model.add(Activation('softmax'))         # output class probabilities

model.summary()

Model: "sequential_15"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_29 (Dense)             (None, 32)                24032     
_________________________________________________________________
activation_29 (Activation)   (None, 32)                0         
_________________________________________________________________
dense_30 (Dense)             (None, 4)                 132       
_________________________________________________________________
activation_30 (Activation)   (None, 4)                 0         
Total params: 24,164
Trainable params: 24,164
Non-trainable params: 0
_________________________________________________________________


In [235]:
from keras.optimizers import SGD, Adam, Adagrad

optimizer = SGD(lr=0.01) # lr is the learning rate
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [238]:
# Train the model, iterating on the data in batches of 32 samples
model.fit(X_train, y_train, epochs=10, batch_size=50);

Epoch 1/10
8751/8751 [==============================] - 0s 22us/step - loss: 0.9671 - accuracy: 0.5916
Epoch 2/10
8751/8751 [==============================] - 0s 22us/step - loss: 0.9649 - accuracy: 0.5920
Epoch 3/10
8751/8751 [==============================] - 0s 22us/step - loss: 0.9627 - accuracy: 0.5916
Epoch 4/10
8751/8751 [==============================] - 0s 22us/step - loss: 0.9607 - accuracy: 0.5923
Epoch 5/10
8751/8751 [==============================] - 0s 22us/step - loss: 0.9588 - accuracy: 0.5924
Epoch 6/10
8751/8751 [==============================] - 0s 22us/step - loss: 0.9569 - accuracy: 0.5928
Epoch 7/10
8751/8751 [==============================] - 0s 22us/step - loss: 0.9549 - accuracy: 0.5932
Epoch 8/10
8751/8751 [==============================] - 0s 22us/step - loss: 0.9531 - accuracy: 0.5938
Epoch 9/10
8751/8751 [==============================] - 0s 22us/step - loss: 0.9511 - accuracy: 0.5940
Epoch 10/10
8751/8751 [==============================] - 0s 22us/step - l

In [239]:
# evaluate the keras model 
_, accuracy = model.evaluate(X_test, y_test)
print('Accuracy: %.2f' % (accuracy*100))

4311/4311 [==============================] - 0s 14us/step
Accuracy: 59.08


In [240]:
# Or better to making predictions and get the confusion matrix
from sklearn import metrics
y_pred = model.predict(X_test)
print(y_pred)

print('confusion matrix: ')
matrix = metrics.confusion_matrix(y_test.argmax(axis=1), y_pred.argmax(axis=1))
print(matrix)

[[0.08131247 0.5582084  0.34116635 0.01931274]
 [0.09584083 0.5326751  0.34417722 0.02730682]
 [0.0999978  0.5618301  0.3121924  0.02597976]
 ...
 [0.08508591 0.6176477  0.26785296 0.02941343]
 [0.08209634 0.5953569  0.28649896 0.03604779]
 [0.08066204 0.5804497  0.3179315  0.02095677]]
confusion matrix: 
[[   0  366    6    0]
 [   0 2539    8    0]
 [   0 1275    8    0]
 [   0  109    0    0]]


In [249]:
sample = pd.read_csv('./data/sampleSubmission.csv')
print(sample.head())

test_a = pca.transform(test)

y_pred = model.predict(test_a)

decoded_y_pred = []

# Code adapted from https://jovianlin.io/keras-one-hot-encode-decode-sequence-data/
#----------------------------------------

def decode(datum):
    return np.argmax(datum)

for i in range(y_pred.shape[0]):
    datum = y_pred[i]
    decoded_datum = decode(y_pred[i])
    decoded_y_pred.append(decoded_datum)
    
#---------------------------------------    
    
decoded_y_pred = le.inverse_transform(decoded_y_pred)

results = pd.DataFrame()
results["ID"]=sample["ID"]
results["Predicted"]=decoded_y_pred

#Export result
results.to_csv("submissionNN.csv", index=False)

       ID Predicted
0  C00000         N
1  C00001         N
2  C00002         A
3  C00003         N
4  C00004         N
